In [ ]:
# Single Sample PDE Generation and Visualization
# This notebook generates a single sample of a random geometry and solves a PDE on it

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from scipy.fft import fft2, ifft2
from pyiga import bspline, assemble, geometry, vis
import scipy.sparse.linalg
import os
from geniga.geometry.random import generate_random_geom, generate_random_bcs

# Set random seed for reproducibility (optional)
#np.random.seed(42)

print("Imports successful!")


In [ ]:
# Parameters for geometry and PDE
N = (64, 64)  # Grid resolution
deg = 2       # B-spline degree
equation = "helmholtz"  # Type of PDE to solve ("poisson" or "helmholtz")

# Parameters for random geometry generation
epsilon_strain = 4.0
epsilon_rotation = 4.0

# Parameters for boundary conditions
bc_scale = 2.0
bc_sigma = 16.0/2.0

# Parameters for Helmholtz equation (only used if equation="helmholtz")
k = 16.0  # Wave number

print(f"Grid resolution: {N}")
print(f"B-spline degree: {deg}")
print(f"Equation type: {equation}")
if equation == "helmholtz":
    print(f"Wave number k: {k}")
print(f"Strain epsilon: {epsilon_strain}")
print(f"Rotation epsilon: {epsilon_rotation}")


In [ ]:
# Generate random geometry
print("Generating random geometry...")
geo, spline_space, X_new, Y_new = generate_random_geom(
    N=N, 
    deg=deg, 
    epsilon_strain=epsilon_strain, 
    epsilon_rotation=epsilon_rotation
)

print(f"Generated geometry with {geo.coeffs.shape} control points")
print(f"Spline space dimensions: {[kvs.numdofs for kvs in spline_space]}")

# Display some information about the geometry
print(f"X coordinates range: [{X_new.min():.3f}, {X_new.max():.3f}]")
print(f"Y coordinates range: [{Y_new.min():.3f}, {Y_new.max():.3f}]")


In [ ]:
# Generate boundary conditions
print("Generating boundary conditions...")
bcs = generate_random_bcs(geo, spline_space, scale=bc_scale, sigma=bc_sigma)

print(f"Number of Dirichlet boundary points: {len(bcs[0])}")
print(f"Boundary values range: [{bcs[1].min():.3f}, {bcs[1].max():.3f}]")

# The bcs object contains:
# bcs[0]: indices of boundary degrees of freedom
# bcs[1]: values at those boundary points


In [ ]:
# Solve the PDE
print("Setting up and solving the PDE...")

# Define the right-hand side (source term) - in this case, zero
rhs = assemble.inner_products(spline_space, lambda x, y: 0 * x, f_physical=True, geo=geo).ravel()

if equation == "poisson":
    # Assemble stiffness matrix for Poisson equation: -∇²u = f
    A = assemble.stiffness(spline_space, geo)
    print(f"Stiffness matrix shape: {A.shape}")
    print(f"Matrix sparsity: {A.nnz / (A.shape[0] * A.shape[1]):.4f}")
    
    # Create restricted linear system with boundary conditions
    LS = assemble.RestrictedLinearSystem(A, rhs, bcs)
    print(f"Restricted system size: {LS.A.shape}")
    
    # Solve the linear system
    u_restricted = scipy.sparse.linalg.spsolve(LS.A, LS.b)
    u_full = LS.complete(u_restricted)
    
    print(f"Solution computed successfully!")
    print(f"Solution range: [{u_full.min():.6f}, {u_full.max():.6f}]")

elif equation == "helmholtz":
    # Assemble matrices for Helmholtz equation: -∇²u + k²u = f
    A = assemble.stiffness(spline_space, geo)  # Stiffness matrix (-∇² term)
    M = assemble.mass(spline_space, geo)       # Mass matrix (k²u term)
    print(f"Stiffness matrix shape: {A.shape}")
    print(f"Mass matrix shape: {M.shape}")
    print(f"Stiffness matrix sparsity: {A.nnz / (A.shape[0] * A.shape[1]):.4f}")
    print(f"Mass matrix sparsity: {M.nnz / (M.shape[0] * M.shape[1]):.4f}")
    print(f"Wave number k: {k}")
    
    # Combine matrices: -A + k²M (note: stiffness matrix already has negative sign)
    combined_matrix = -A + k**2 * M
    
    # Create restricted linear system with boundary conditions
    LS = assemble.RestrictedLinearSystem(combined_matrix, rhs, bcs)
    print(f"Restricted system size: {LS.A.shape}")
    
    # Solve the linear system
    u_restricted = scipy.sparse.linalg.spsolve(LS.A, LS.b)
    u_full = LS.complete(u_restricted)
    
    print(f"Solution computed successfully!")
    print(f"Solution range: [{u_full.min():.6f}, {u_full.max():.6f}]")
    
else:
    raise ValueError(f"Unknown equation: {equation}")

print(u_full.shape, geo.coeffs.shape)
# Create B-spline function from solution
u_func = geometry.BSplineFunc(spline_space, u_full)


In [ ]:
# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 12

# Plot 1: Geometry (mesh)
plt.figure(figsize=(8, 6))
plt.title('Generated Random Geometry', fontsize=14, fontweight='bold')
vis.plot_geo(geo, grid=32)
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.grid(True, alpha=0.3)
plt.axis('equal')

# Save geometry plot
geometry_filename = "geometry.png"
plt.savefig(geometry_filename, dpi=300, bbox_inches='tight')
print(f"Geometry plot saved as: {geometry_filename}")
plt.show()

# Plot 2: Solution field
plt.figure(figsize=(8, 6))
equation_title = f"{equation.capitalize()} Equation"
if equation == "helmholtz":
    equation_title += f" (k={k})"
plt.title(f'{equation_title} Solution Field', fontsize=14, fontweight='bold')
vis.plot_field(u_func, geo)
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.colorbar()
plt.axis('equal')

# Save solution plot
solution_filename = f"solution_{equation}.png"
plt.savefig(solution_filename, dpi=300, bbox_inches='tight')
print(f"Solution plot saved as: {solution_filename}")
plt.show()


In [ ]:
# Residual check - verify the solution accuracy
print("\n" + "="*50)
print("RESIDUAL CHECK")
print("="*50)

# Check the norm of the restricted system residual
system_residual_norm = scipy.linalg.norm(LS.A @ u_restricted - LS.b)
print(f"Norm of restricted system residual: {system_residual_norm:.2e}")

# Create masks for boundary and interior points
# Assuming square grid structure for spline space
n0, n1 = [kvs.numdofs for kvs in spline_space]
print(f"Spline space dimensions: {n0} x {n1}")

# Create boundary mask (1 for boundary points, 0 for interior)
bd_mask = np.ones([n0, n1])
bd_mask[1:-1, 1:-1] = 0  # Interior points are 0
bd_mask = bd_mask.flatten()

# Create interior mask (complement of boundary mask)
int_mask = 1 - bd_mask
int_mask = int_mask.flatten()

# Create projection matrices
Pin = scipy.sparse.diags(int_mask, 0, shape=(n0*n1, n0*n1), format='csc')
Pbd = scipy.sparse.diags(bd_mask, 0, shape=(n0*n1, n0*n1), format='csc')

# Get full solution coefficients
uc = u_full.flatten()

if equation == "poisson":
    # For Poisson equation: -∇²u = f
    # Residual: Pin@(A@(Pin@uc)) - Pin@(-A@(Pbd@uc)) - Pin@rhs
    res = Pin@(A@(Pin@uc)) - Pin@(-A@(Pbd@uc)) - Pin@rhs
    
elif equation == "helmholtz":
    # For Helmholtz equation: (-∇² + k²)u = f
    # Use the combined matrix
    res = Pin@(combined_matrix@(Pin@uc)) - Pin@(-combined_matrix@(Pbd@uc)) - Pin@rhs

residual_norm = scipy.linalg.norm(res, ord=np.inf)
print(f"Norm of PDE residual (infinity norm): {residual_norm:.2e}")

# Additional diagnostics
print(f"\nSolution statistics:")
print(f"  - Full solution shape: {u_full.shape}")
print(f"  - Solution range: [{u_full.min():.6f}, {u_full.max():.6f}]")
print(f"  - Solution norm (L2): {scipy.linalg.norm(u_full):.6f}")
print(f"  - Number of boundary DOFs: {bd_mask.sum()}")
print(f"  - Number of interior DOFs: {int_mask.sum()}")

# Check if solution is reasonable
if residual_norm < 1e-10:
    print(f"\n✓ Solution appears to be accurate (residual < 1e-10)")
elif residual_norm < 1e-6:
    print(f"\n✓ Solution appears to be reasonably accurate (residual < 1e-6)")
else:
    print(f"\n⚠ Warning: Large residual detected (residual = {residual_norm:.2e})")
    print("  This might indicate numerical issues or convergence problems.")

print("="*50)
